# Synthetic Optical Flow from Fused Lidar


In [1]:
import sys
sys.path.append('/opt/psegs')

import numpy as np

from psegs.exp.fused_lidar import FusedLidarCloudTableBase

## SemanticKITTI

In [2]:
from psegs.exp.semantic_kitti import SemanticKITTISDTable
class SemanticKITTIFusedWorldCloudTable(FusedLidarCloudTableBase):
    SRC_SD_TABLE = SemanticKITTISDTable
    
    @classmethod
    def _should_build_obj_clouds(cls, segment_uri):
        # SemanticKITTI has no cuboids, so we skip this step.
        return False
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        seg_rdd = cls.SRC_SD_TABLE.get_segment_datum_rdd(spark, segment_uri)
        
        # SemanticKITTI has no cuboids, so the Fuser algo simply concats the cloud points
        def iter_task_rows(iter_sds):
            from pyspark import Row
            from oarphpy.spark import RowAdapter
            for sd in iter_sds:
                if sd.point_cloud is not None:
                    pc = sd.point_cloud
                    task_id = "%s.%s" % (sd.uri.segment_id, pc.extra['semantic_kitti.scan_id'])
                    yield Row(
                        task_id=task_id,
                        point_clouds=[pc],
                        cuboids=[])
        
        task_rdd = seg_rdd.mapPartitions(iter_task_rows)
        return task_rdd
        


## KITTI-360

In [3]:
from psegs.datasets.kitti_360 import KITTI360SDTable
class KITTI360OurFusedClouds(KITTI360SDTable):
    INCLUDE_FISHEYES = False
    INCLUDE_FUSED_CLOUDS = False  # Use our own fused clouds

class KITTI360OurFusedWorldCloudTable(FusedLidarCloudTableBase):
    SRC_SD_TABLE = KITTI360OurFusedClouds
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
        datum_df.registerTempTable('datums')
        spark.sql("""
          CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
          SELECT 
              CONCAT(uri.segment_id, '.', uri.extra.`kitti-360.frame_id`) AS task_id,
              FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
              COLLECT_LIST(point_cloud) AS point_clouds
          FROM datums
          WHERE 
              uri.topic LIKE '%cuboid%' OR uri.topic LIKE '%lidar%'
          GROUP BY task_id
        """)
        tasks_df = spark.sql('SELECT * FROM culi_tasks_df')
        return tasks_df.rdd


## NuScenes

In [2]:
!pip3 install nuscenes-devkit==1.1.2
from psegs.datasets.nuscenes import NuscStampedDatumTableBase
class NuscFusedWorldCloudTable(FusedLidarCloudTableBase):
    SRC_SD_TABLE = NuscStampedDatumTableBase
    
    SPLITS = ['train_detect', 'train_track']
    
    @classmethod
    def _filter_ego_vehicle(cls, cloud_ego):
        cloud_ego = cloud_ego[np.where(  ~(
                        (cloud_ego[:, 0] <= 1.5) & (cloud_ego[:, 0] >= -1.5) &  # Nusc lidar +x is +right
                        (cloud_ego[:, 1] <= 2.5) & (cloud_ego[:, 0] >= -2.5) &  # Nusc lidar +y is +forward
                        (cloud_ego[:, 1] <= 1.5) & (cloud_ego[:, 0] >= -1.5)   # Nusc lidar +z is +up
        ))]
        return cloud_ego
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
        datum_df.registerTempTable('datums')
        
        if cls.SRC_SD_TABLE.LABELS_KEYFRAMES_ONLY:
            # For Nusc: group by nuscenes-sample-token WITH KEYFRAMES
            spark.sql("""
              CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
              SELECT 
                  uri.extra.`nuscenes-sample-token` AS task_id,
                  FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
                  COLLECT_LIST(point_cloud) AS point_clouds
              FROM datums
              WHERE 
                uri.extra.`nuscenes-is-keyframe` = 'True' AND (
                  uri.extra['nuscenes-label-channel'] is NULL OR 
                  uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
                ) AND (
                  uri.topic LIKE '%cuboid%' OR
                  uri.topic LIKE '%lidar%'
                )
              GROUP BY task_id
            """)
        else:
            # For Nusc: group by nuscenes-sample-token WITH ALL FRAMES
            spark.sql("""
              CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
              SELECT 
                  CONCAT(uri.segment_id, '.', uri.timestamp) AS task_id,
                  FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
                  COLLECT_LIST(point_cloud) AS point_clouds
              FROM datums
              WHERE 
                (
                  uri.extra['nuscenes-label-channel'] is NULL OR 
                  uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
                ) AND (
                  uri.topic LIKE '%cuboid%' OR
                  uri.topic LIKE '%lidar%'
                )
              GROUP BY task_id
              HAVING SIZE(cuboids) > 0 AND SIZE(pcs) > 0
            """)
        
        tasks_df = spark.sql('SELECT * FROM culi_tasks_df')
        return tasks_df.rdd


     |████████████████████████████████| 282 kB 16.0 MB/s eta 0:00:01
     |████████████████████████████████| 87 kB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 30.8 MB/s eta 0:00:01
  Using cached Cython-0.29.21-cp38-cp38-manylinux1_x86_64.whl (1.9 MB)
     |████████████████████████████████| 119 kB 77.2 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 5.8 MB/s  eta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp38-cp38-linux_x86_64.whl size=417538 sha256=ef2c6bb9201cd102b7b55286cc51be2411322964ec7f1d878490018f6a20bec2
  Stored in directory: /root/.cache/pip/wheels/e7/77/b2/6f38b5bea571cd8f4689f91a7c1ed2eaecb2c2ce17f9945b17


  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=250a287c72334853836b4f2562fbbae1d0db70c960e555efb641af473db607d9
  Stored in directory: /root/.cache/pip/wheels/1f/10/06/2a990ee4d73a8479fe2922445e8a876d38cfbfed052284c6a1
Successfully built pycocotools fire


## Start Spark

In [3]:
from psegs.spark import NBSpark
spark = NBSpark.getOrCreate()

2021-02-13 03:26:04,550	oarph 48 : Using source root /opt/psegs/psegs 
2021-02-13 03:26:04,551	oarph 48 : Using source root /opt/psegs 
2021-02-13 03:26:04,620	oarph 48 : Generating egg to /tmp/tmpkk0zf6m2_oarphpy_eggbuild ...
2021-02-13 03:26:04,702	oarph 48 : ... done.  Egg at /tmp/tmpkk0zf6m2_oarphpy_eggbuild/psegs-0.0.0-py3.8.egg


## Build Fused Lidar Assets

```
docker --context default run -it --name=potree_viewer --rm --net=host -v `pwd`:/shared  jonazpiazu/potree
```

In [ ]:
seg_uris = NuscFusedWorldCloudTable.get_all_segment_uris()
sd_rdd = NuscFusedWorldCloudTable._get_segment_datum_rdd_or_df(spark, seg_uris[0])
print(sd_rdd.count())

2021-02-13 03:26:10,155	ps   48 : Creating shelve caches; will use ~8-10GB of RAM ...
2021-02-13 03:26:56,931	ps   48 : ... NuScenes done loading & indexing JSON data ...
2021-02-13 03:26:56,931	ps   48 : Building shelve cache for category (in /tmp/psegs_temp/nuscenes/v1.0-trainval/category) ...
2021-02-13 03:26:56,948	ps   48 : Building shelve cache for attribute (in /tmp/psegs_temp/nuscenes/v1.0-trainval/attribute) ...
2021-02-13 03:26:56,952	ps   48 : Building shelve cache for visibility (in /tmp/psegs_temp/nuscenes/v1.0-trainval/visibility) ...
2021-02-13 03:26:56,955	ps   48 : Building shelve cache for instance (in /tmp/psegs_temp/nuscenes/v1.0-trainval/instance) ...
2021-02-13 03:26:57,589	ps   48 : Building shelve cache for sensor (in /tmp/psegs_temp/nuscenes/v1.0-trainval/sensor) ...
2021-02-13 03:26:57,592	ps   48 : Building shelve cache for calibrated_sensor (in /tmp/psegs_temp/nuscenes/v1.0-trainval/calibrated_sensor) ...
2021-02-13 03:26:57,691	ps   48 : Building shelve cac